### 07.19 晴天　心情好


### 1. Iris 数据集学习

In [8]:
from sklearn import datasets
iris = datasets.load_iris()

print(iris.data.shape)

(150, 4)


In [9]:
#输出数据全部
#print(iris.data[:])
print(iris.data[:5])

[[5.1 3.5 1.4 0.2]
 [4.9 3.  1.4 0.2]
 [4.7 3.2 1.3 0.2]
 [4.6 3.1 1.5 0.2]
 [5.  3.6 1.4 0.2]]


In [14]:
print(iris.target.view())
print(iris.target_names)
print(iris.feature_names)

[0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 2 2 2 2 2 2 2 2 2 2
 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2
 2 2]
['setosa' 'versicolor' 'virginica']
['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']


### 2. 算法实践

In [36]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split

from collections import Counter
import math

# data
def creat_data():
    iris = load_iris()
    df = pd.DataFrame(iris.data, columns=iris.feature_names)
    df["label"] = iris.target
    df.columns = ["sepal length", "sepal width", "petal length","petal width", "label"]
    print(df.shape)
    data = np.array(df.iloc[:100, :])#加载数据集前100个　标签为0,1
    #print(data)
    return data[:, :-1], data[:, -1]#返回标数据和标签

X, y = creat_data()
#随机分离数据集函数
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)


In [56]:
#查看数据集
X_test[0], y_test[0]

(array([5.5, 2.4, 3.8, 1.1]), 1.0)

In [146]:
class NaiveBayes:
    
    def __init__(self):
        self.model = None
        
    #数学期望
    @staticmethod
    def mean(X):
        return sum(X)/float(len(X))
    
    #标准差
    def stdev(self, X):
        avg = self.mean(X)
        return math.sqrt(sum([pow(x-avg, 2) for x in X])/float(len(X)))
        
    #概率密度函数
    def gaussian_probability(self, x, mean, stdev):
        exponent = math.exp(-(math.pow(x- mean, 2)/(2*math.pow(stdev, 2))))
        
        return 1 / math.sqrt(2*math.pi*math.pow(stdev, 2))*exponent
    
    #处理X_train数据,得到列向量的均值和方差
    def summarize(self, train_data):
        #print(train_data)
#         for i in zip(*train_data):
#             print(i)
        summaries = [(self.mean(i), self.stdev(i)) for i in zip(*train_data)]#取出每一个数行的z
        #print(summaries)
        return summaries
    
    #分别求出期望和方差
    def fit(self, X, y):
        labels = list(set(y))#set()构建一个不重复的集合
        print("labels:",labels)
        #封装数据为为字典
        data = {label : [] for label in labels}
        
        for f, label in zip(X, y): 
            data[label].append(f)#添加数据字典
            
        self.model = {
            label: self.summarize(value) for label, value in data.items()
        }
        print("model:", self.model)
        return 'gaussian NB train done!'
    
    #计算概率
    def calculate_probabilities(self, input_data):
        probabilities = {}
        for label,value in self.model.items():
            probabilities[label]=1#保证非零
            print("value:", value)
            for i in range(len(value)):
                #print(i," : ",value[i])
                mean, stdev = value[i]
                probabilities[label] *= self.gaussian_probability(
                input_data[i], mean, stdev)
          
        print("可能:",probabilities)
        return probabilities

    # 类别
    def predict(self, X_test):
        # {0.0: 2.9680340789325763e-27, 1.0: 3.5749783019849535e-26}
        label = sorted(
            self.calculate_probabilities(X_test).items(),
            key=lambda x: x[-1])[-1][0]
        return label

    def score(self, X_test, y_test):
        right = 0
        for X, y in zip(X_test, y_test):
          
            label = self.predict(X)  
            print("label:",label)
            print("True:", y)
            if label == y:
                right += 1


                return right / float(len(X_test))

In [147]:
model = NaiveBayes()

In [148]:
model.fit(X_train, y_train)

labels: [0.0, 1.0]
model: {0.0: [(5.0, 0.364887458187942), (3.4371428571428573, 0.30899441907647485), (1.465714285714286, 0.17061712716381275), (0.2371428571428572, 0.10713333291001527)], 1.0: [(5.902857142857142, 0.4999918366680539), (2.777142857142857, 0.2928536585158575), (4.2457142857142856, 0.4776985643346094), (1.3228571428571425, 0.19432772655007943)]}


'gaussian NB train done!'

In [149]:
print(model.predict([4.4, 3.2, 1.3, 0.2]))

value: [(5.0, 0.364887458187942), (3.4371428571428573, 0.30899441907647485), (1.465714285714286, 0.17061712716381275), (0.2371428571428572, 0.10713333291001527)]
value: [(5.902857142857142, 0.4999918366680539), (2.777142857142857, 0.2928536585158575), (4.2457142857142856, 0.4776985643346094), (1.3228571428571425, 0.19432772655007943)]
可能: {0.0: 1.391870274889773, 1.0: 2.232200710682964e-18}
0.0


In [150]:
model.score(X_test, y_test)

value: [(5.0, 0.364887458187942), (3.4371428571428573, 0.30899441907647485), (1.465714285714286, 0.17061712716381275), (0.2371428571428572, 0.10713333291001527)]
value: [(5.902857142857142, 0.4999918366680539), (2.777142857142857, 0.2928536585158575), (4.2457142857142856, 0.4776985643346094), (1.3228571428571425, 0.19432772655007943)]
可能: {0.0: 3.189024994637145e-57, 1.0: 0.1970620443735743}
label: 1.0
True: 1.0


0.03333333333333333

### 3. scikit-learn实例


In [151]:
from sklearn.naive_bayes import  GaussianNB


In [152]:
clf = GaussianNB()
clf.fit(X_train,y_train)

GaussianNB(priors=None)

In [154]:
clf.score(X_test, y_test)

1.0

In [158]:
clf.predict([[1],[5],[5],[4],[5.3],[1.2]])


array([1., 1., 1., 1., 1., 1.])

In [157]:
from sklearn.naive_bayes import BernoulliNB, MultinomialNB # 